<a href="https://colab.research.google.com/github/yoonandmoon/GS_team/blob/main/231018_%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#실습2_리뷰데이터로 토픽모델링

Q) 부정 리뷰 data에 몇 가지 종류의 토픽이 있는지 LDA모형을 통해 추출해 보세요.

1) 데이터토큰화

2) [선택]불용어 처리-> 단어 기반/pos-tag 기반

3) TF-IDF(TDM) 만든 후 -> LDA 모델링

4) pyLDAvis로 시각화하여 몇 개의 토픽으로 분류하는 것이 가장 적합할지 추론





In [ ]:
import warnings
#경고메세지 끄기
warnings.filterwarnings('ignore')

#참고) 다시 출력하게 하기
#warnings.filterwarnings(actions='default')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/review.csv',sep = '\t')



#토큰화 및 데이터 전처리

In [ ]:
#pip install konlpy

In [ ]:
# 코모란
from konlpy.tag import Okt
#선언
tokenizer = Okt()
# 토큰화 : morphs
df['morphs'] = None
for i, row in df.iterrows():
  df['morphs'][i] = ' '.join(tokenizer.morphs(row['document']))

In [ ]:
df.head()

,id,document,label,morphs
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 .. 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 ... 포스터 보고 초딩 영화 줄 .... 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 .. 솔직히 재미 는 없다 .. 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 ! 스파이더맨 에서 늙어 보이기만...


In [ ]:
import re
# 한글을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]')

def preprocess_remove(text):
  text = re.sub(remove_except_ko,' ',text).strip()
  return text
df['morphs'] = df['morphs'].map(lambda x : preprocess_remove(x))
df.head()

,id,document,label,morphs
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...


In [ ]:
#불용어처리
stop_word = ['포스터','저작권자','한경','닷컴','뉴스룸','홈페이지','바로','네이버','채널','구독','세요','제공'
,'께서','라고','입니다','습니다','한다','에서','이다','에게','으로','이랑','까지','부터','하다','한데','통해','위해','때문']

def preprocess(text):
  text = text.split()
  text = [i for i in text if len(i)>1]
  text = [i for i in text if i not in stop_word]
  return text

In [ ]:
text = '최윤선 기자 안녕하세요 공'
preprocess(text)

['최윤선', '기자', '안녕하세요']

In [ ]:
#토큰화 + 토큰리스트 생성
def make_tokens(df):
  df['tokens'] = None
  for i,row in df.iterrows():
    if i%1000==0:
      print(i,'/',len(df))
    token = preprocess(df['morphs'][i])
    df['tokens'][i]= ' '.join(token)
  return df
df = make_tokens(df)

0 / 10000
1000 / 10000
2000 / 10000
3000 / 10000
4000 / 10000
5000 / 10000
6000 / 10000
7000 / 10000
8000 / 10000
9000 / 10000


In [ ]:
neg = (df[df['label']==0]).reset_index(drop = True)
pos= (df[df['label']==1]).reset_index(drop = True)
print('neg:',len(neg))
print('pos:',len(pos))

neg: 5021
pos: 4979


In [ ]:
df.head()

,id,document,label,morphs,tokens
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리,더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나,보고 초딩 영화 오버 연기 조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너 무재 밓었 다그 래서 보는것을 추천 한 다,무재 밓었 다그 래서 보는것을 추천
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정,교도소 이야기 구먼 솔직히 재미 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만...,사이 몬페 익살스런 연기 돋보였던 영화 스파이더맨 늙어 보이기만 했던 커스틴 던스트...


In [ ]:
from gensim import corpora
from gensim.models import LdaModel,TfidfModel

tokenized_docs = neg['tokens'].apply(lambda x : x.split())

id2word=corpora.Dictionary(tokenized_docs)
corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs]
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 #토픽개수(변경가능)
lda = LdaModel(corpus=corpus_TFIDF,
               id2word=id2word,
               num_topics=n,
               random_state=100)
for t in lda.print_topics():
  print(t)

(0, '0.007*"없다" + 0.005*"영화" + 0.004*"지루하다" + 0.004*"하나" + 0.004*"기세" + 0.003*"역시" + 0.003*"눈물" + 0.003*"재미" + 0.003*"진짜" + 0.003*"시간"')
(1, '0.012*"영화" + 0.004*"결말" + 0.004*"별루" + 0.003*"소설" + 0.003*"였다" + 0.003*"그닥" + 0.003*"관객" + 0.003*"너무" + 0.003*"그만" + 0.003*"별로"')
(2, '0.006*"ㅋㅋ" + 0.004*"영화" + 0.004*"인지" + 0.004*"나옴" + 0.004*"노잼" + 0.003*"스토리" + 0.003*"마지막" + 0.003*"그냥" + 0.003*"대사" + 0.003*"개봉"')
(3, '0.005*"이딴" + 0.004*"영화" + 0.003*"같은" + 0.003*"아무" + 0.003*"마무리" + 0.002*"런가" + 0.002*"쓰래" + 0.002*"극치" + 0.002*"영화로" + 0.002*"재미없"')
(4, '0.009*"영화" + 0.008*"최악" + 0.004*"쓰레기" + 0.004*"만든" + 0.004*"처음" + 0.004*"진짜" + 0.003*"정말" + 0.003*"후회" + 0.003*"졸작" + 0.003*"이런"')
(5, '0.009*"영화" + 0.006*"재미없다" + 0.006*"너무" + 0.004*"이런" + 0.004*"시간" + 0.004*"없이" + 0.004*"평점" + 0.003*"점도" + 0.003*"ㅡㅡ" + 0.003*"무슨"')
(6, '0.005*"이건" + 0.005*"영화" + 0.005*"진짜" + 0.004*"평점" + 0.004*"재미없는" + 0.003*"ㅋㅋㅋㅋ" + 0.003*"아깝다" + 0.003*"재미없어" + 0.003*"없는" + 0.003*"노답"')
(7, '0.006*"영화" + 0.004*"코믹" + 0.004*"

시각화

In [ ]:
# !pip install pyLDAvis==3.4.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda,corpus_TFIDF,id2word)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


BrokenProcessPool: ignored

In [ ]:
from gensim import corpora
from gensim.models import LdaModel,TfidfModel

tokenized_docs = neg['tokens'].apply(lambda x : x.split())

id2word=corpora.Dictionary(tokenized_docs)
corpus_TDM = [id2word.doc2bow(doc) for doc in tokenized_docs]
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 3 #토픽개수(변경가능)
lda = LdaModel(corpus=corpus_TFIDF,
               id2word=id2word,
               num_topics=n,
               random_state=100)
for t in lda.print_topics():
  print(t)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda,corpus_TFIDF,id2word)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.005*"영화" + 0.004*"없다" + 0.003*"이런" + 0.003*"진짜" + 0.003*"시간" + 0.003*"재미" + 0.003*"너무" + 0.002*"이건" + 0.002*"재미없다" + 0.002*"평점"')
(1, '0.007*"영화" + 0.003*"별로" + 0.002*"아깝다" + 0.002*"너무" + 0.002*"스토리" + 0.002*"감독" + 0.002*"만든" + 0.002*"정말" + 0.002*"없는" + 0.002*"하는"')
(2, '0.004*"쓰레기" + 0.004*"영화" + 0.003*"최악" + 0.002*"그냥" + 0.002*"진짜" + 0.002*"보다" + 0.002*"스토리" + 0.002*"ㅡㅡ" + 0.002*"같은" + 0.002*"ㅋㅋ"')


BrokenProcessPool: ignored

In [ ]:
#KnuSentiLex 다운로드
!wget https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/pos_pol_word.txt
!wget https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/neg_pol_word.txt

dct = {}

# 긍정단어 조회
with open("pos_pol_word.txt", "r") as f :
  dct['pos'] = f.read().split("\n")[19:]

# 부정단어 조회
with open("neg_pol_word.txt", "r") as f :
  dct['neg'] = f.read().split("\n")[19:]

print(dct['pos'][:10])
print(dct['neg'][:10])

--2023-10-18 08:38:37--  https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/pos_pol_word.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85007 (83K) [text/plain]
Saving to: ‘pos_pol_word.txt.2’

pos_pol_word.txt.2  100%[===================>]  83.01K  --.-KB/s    in 0.02s   

2023-10-18 08:38:37 (3.83 MB/s) - ‘pos_pol_word.txt.2’ saved [85007/85007]

--2023-10-18 08:38:37--  https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/neg_pol_word.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176542 (172

In [ ]:
print(len(dct['neg'])),print(len(dct['pos']))

9828
4869


(None, None)

In [ ]:
txt = '코로나19의 여파로 전 세계 교사들과 학생들 모두 혼란스러운 시기를 보내고 있습니다. 초유의 상황에서도, 학생들을 위해 최선을 다하려는 선생님들의 노력이 계속되고 있는데요. 어려움 속에서, 선생님들은 어떤 방법으로 사랑을 전하고 있을까요? 뉴스G에서 전해드립니다. [리포트] 중국 저장성의 한 산골 마을. 초등학교 교사 왕진량 씨는 지난 2월 말부터, 매일 새벽 다섯 시면 부지런히 집을 나섭니다. 온라인 수업을 받을 수 없는, 깊은 산골 마을에 살고 있는 학생들을 찾아가기 위해서인데요. 선생님이 하루 동안 이동하는 거리는 대략 30km정도. 차도 없이 도보로 네 개의 마을을 돌아다니며 학생들을 만납니다. 매일 이어지는 강행군이지만, 오로지 아이들의 학습이 중단되어서는 안 된다는 생각뿐입니다. 혹시 모를 사태에 대비해 학생들과의 접촉은 최대한 줄입니다. 숙제를 내주고, 검사 후에 모르는 문제를 알려주는 식으로 일대일 수업을 진행하고 있는데요. 아이들에게 배우는 즐거움이 얼마나 소중한 것인지 잘 알기에, 선생님은 이렇게라도 수업을 할 수 있다는 데서 행복을 느낍니다. 영국의 한 초등학교 교사인 젠 포울스 씨는 매일 아침, 무거운 짐을 앞 뒤, 양 옆으로 짊어지고 씩씩하게 발걸음을 옮깁니다. 코로나19로 학교가 문을 닫은 뒤, 형편이 어려운 학생들을 위해 매일 78인분의 점심 도시락을 배달하고 있는데요. 선생님이 재직 중인 초등학교는 전체 학생의 41퍼센트가 무상 급식 대상자이기 때문입니다. 도시락의 무게는 18kg, 걸어야 하는 거리는 8km에 달하지만 기다리는 학생들을 생각하며 지치지 않고 발걸음을 재촉합니다. 학생들은 창문을 통해서 반갑게 인사하기도 하고, 선생님이 볼 수 있게 감사 메시지를 붙여 놓기도 하는데요. 선생님이 정성껏 준비해 손수 배달한 사랑의 도시락. 봉쇄된 도시의 굶주린 아이들에게 소중한 한 끼 식사, 그 이상의 의미가 되고 있습니다. 미국 사우스다코타 주의 중학교 수학 교사인 크리스 와바 씨는, 커다란 화이트보드를 들고 학생의 집을 찾았습니다. 온라인 수업 후, 학생에게 이메일로 방정식 풀이법에 대한 질문을 받았기 때문인데요. 이메일로 답변해주는 것보다 직접 풀이 과정을 보여주는 게 낫다는 생각이었죠. 깜짝 놀란 학생을 마주한 채, 선생님은 현관문 앞에서 열정적으로 문제를 풀기 시작했습니다. 이 열정적인 강의는, 학생이 풀이법을 완벽히 이해할 때까지 이어졌는데요. 바이러스는 전 세계 교실에 혼란을 불러왔지만, 선생님들의 노력은 저마다의 방식으로 계속되고 있습니다. 어려움 속에서도 학생들을 위해 안간힘을 쓰고 있는 모든 선생님들에게, 응원과 박수를 함께 보냅니다.'
txt

'코로나19의 여파로 전 세계 교사들과 학생들 모두 혼란스러운 시기를 보내고 있습니다. 초유의 상황에서도, 학생들을 위해 최선을 다하려는 선생님들의 노력이 계속되고 있는데요. 어려움 속에서, 선생님들은 어떤 방법으로 사랑을 전하고 있을까요? 뉴스G에서 전해드립니다. [리포트] 중국 저장성의 한 산골 마을. 초등학교 교사 왕진량 씨는 지난 2월 말부터, 매일 새벽 다섯 시면 부지런히 집을 나섭니다. 온라인 수업을 받을 수 없는, 깊은 산골 마을에 살고 있는 학생들을 찾아가기 위해서인데요. 선생님이 하루 동안 이동하는 거리는 대략 30km정도. 차도 없이 도보로 네 개의 마을을 돌아다니며 학생들을 만납니다. 매일 이어지는 강행군이지만, 오로지 아이들의 학습이 중단되어서는 안 된다는 생각뿐입니다. 혹시 모를 사태에 대비해 학생들과의 접촉은 최대한 줄입니다. 숙제를 내주고, 검사 후에 모르는 문제를 알려주는 식으로 일대일 수업을 진행하고 있는데요. 아이들에게 배우는 즐거움이 얼마나 소중한 것인지 잘 알기에, 선생님은 이렇게라도 수업을 할 수 있다는 데서 행복을 느낍니다. 영국의 한 초등학교 교사인 젠 포울스 씨는 매일 아침, 무거운 짐을 앞 뒤, 양 옆으로 짊어지고 씩씩하게 발걸음을 옮깁니다. 코로나19로 학교가 문을 닫은 뒤, 형편이 어려운 학생들을 위해 매일 78인분의 점심 도시락을 배달하고 있는데요. 선생님이 재직 중인 초등학교는 전체 학생의 41퍼센트가 무상 급식 대상자이기 때문입니다. 도시락의 무게는 18kg, 걸어야 하는 거리는 8km에 달하지만 기다리는 학생들을 생각하며 지치지 않고 발걸음을 재촉합니다. 학생들은 창문을 통해서 반갑게 인사하기도 하고, 선생님이 볼 수 있게 감사 메시지를 붙여 놓기도 하는데요. 선생님이 정성껏 준비해 손수 배달한 사랑의 도시락. 봉쇄된 도시의 굶주린 아이들에게 소중한 한 끼 식사, 그 이상의 의미가 되고 있습니다. 미국 사우스다코타 주의 중학교 수학 교사인 크리스 와바 씨는, 커다란 화이트보드를 들고 학생의 집을 찾았습니다

In [ ]:
def analye_sentiment(txt,dct):
  pos=[]
  neg=[]

  for p in dct['pos']:
    if p in txt:
      pos.append(p)
  for n in dct['neg']:
    if n in txt:
      neg.append(n)
  return(len(pos)/(len(pos)+len(neg)),pos),(len(neg)/(len(pos)+len(neg)),neg)

In [ ]:
pos,neg = analye_sentiment(txt,dct)
print(pos)
print(neg)

(0.6470588235294118, ['낫다', '대상', '사랑을', '씩씩하게', '열정', '운', '잘', '정', '정성껏', '함께', '감사', '복을', '부지런', '부지런히', '사랑', '사랑의', '소중한', '즐거움', '즐거움이', '행복', '행복을', ''])
(0.35294117647058826, ['방정', '어려운', '어려움', '해', '혼란스러운', '화', '굶주린', '모르는', '바이러스', '우는', '짐', ''])
